In [1]:
import os, re, json, sys, time, requests, argparse

In [13]:
# def download(user,repo,branch='main',path='src-data'):

#     # cmd = f'git archive --remote=git@github.com:{user}/{repo}.git {branch} {path} | tar -xvf -'
#     cmd = f'wget --recursive --no-parent --no-directories -P ./downloaded-folder/ \
#     --accept "*.*" https://api.github.com/repos/{user}/{repo}/contents/{path}?ref={branch}'
#     print(cmd)
#     os.popen(cmd)
    
    


In [35]:
import os
import requests
from concurrent.futures import ThreadPoolExecutor

def download_file(download_url, file_path):
    """Downloads a single file."""
    try:
        response = requests.get(download_url)
        if response.status_code == 200:
            with open(file_path, "wb") as f:
                f.write(response.content)
            print(f"Downloaded: {file_path}")
        else:
            print(f"Failed to download {file_path}: {response.status_code} - {response.text}")
    except Exception as e:
        print(f"Error downloading {file_path}: {e}")

def process_folder(owner, repo, folder, branch, output_dir):
    """
    Processes a folder: Downloads files and recurses into subdirectories.
    """
    api_url = f"https://api.github.com/repos/{owner}/{repo}/contents/{folder}?ref={branch}"
    response = requests.get(api_url)

    if response.status_code != 200:
        print(f"Failed to fetch folder contents: {response.status_code} - {response.text}")
        return []

    files = response.json()
    tasks = []
    for file in files:
        if file["type"] == "file":
            # File download task
            file_path = os.path.join(output_dir, folder, file["name"])
            os.makedirs(os.path.dirname(file_path), exist_ok=True)
            tasks.append((file["download_url"], file_path))
        elif file["type"] == "dir":
            # Recurse into subdirectory
            subfolder = os.path.join(folder, file["name"])
            tasks.extend(process_folder(owner, repo, subfolder, branch, output_dir))

    return tasks

def download(owner, repo, folder='src-data', branch='main', output_dir='./downloads', max_workers=8):
    """
    Downloads all files from a specific folder in a GitHub repository, including nested directories.
    """
    os.makedirs(output_dir, exist_ok=True)
    tasks = process_folder(owner, repo, folder, branch, output_dir)

    # Use ThreadPoolExecutor to download files in parallel
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(download_file, url, path) for url, path in tasks]
        for future in futures:
            future.result()  # Wait for all downloads to complete

    print("All downloads completed.")

# # Example usage
# if __name__ == "__main__":
#     download(
#         owner="your-username",
#         repo="CRP-UNIVERSE",
#         branch="main",
#         folder="src-data",
#         output_dir="./downloaded-folder",
#         max_workers=8  # Adjust the number of threads for optimal performance
#     )


In [36]:
download('wcrp-cmip','CMIP7_CVs',output_dir='./downloads/CMIP7/')

Downloaded: ./downloads/CMIP7/src-data/.DS_StoreDownloaded: ./downloads/CMIP7/src-data/activity/THESE NOW EXIST IN WCRP UNIVERSE/c4mip.json
Downloaded: ./downloads/CMIP7/src-data/activity/THESE NOW EXIST IN WCRP UNIVERSE/_context

Downloaded: ./downloads/CMIP7/src-data/activity/THESE NOW EXIST IN WCRP UNIVERSE/cfmip.json
Downloaded: ./downloads/CMIP7/src-data/activity/THESE NOW EXIST IN WCRP UNIVERSE/cmip.json
Downloaded: ./downloads/CMIP7/src-data/activity/.DS_Store
Downloaded: ./downloads/CMIP7/src-data/_context_
Downloaded: ./downloads/CMIP7/src-data/activity/THESE NOW EXIST IN WCRP UNIVERSE/aerchemmip.json
Downloaded: ./downloads/CMIP7/src-data/activity/THESE NOW EXIST IN WCRP UNIVERSE/scenariomip.json
Downloaded: ./downloads/CMIP7/src-data/activity/THESE NOW EXIST IN WCRP UNIVERSE/dcpp.json
Downloaded: ./downloads/CMIP7/src-data/activity/THESE NOW EXIST IN WCRP UNIVERSE/damip.json
Downloaded: ./downloads/CMIP7/src-data/activity/THESE NOW EXIST IN WCRP UNIVERSE/geomip.json
Download

NameError: name 'contents' is not defined